# Modelos Preditivos da Análise de Churn

## Contexto do Notebook

Este Notebook tem como objetivo salvar o classificador com o melhor desempenho obtido e documentado neste outro [notebook](https://colab.research.google.com/drive/1JrjWBv1zyUN9tbqNBGXju7EgHMLs5Vcg#scrollTo=QOss5n8yFU_O) e testá-lo para verificar se está prevendo corretamente a maioria dos funcionários, com foco principal na análise de churn.

- ### Detalhes do Notebook

Neste notebook, vou carregar o modelo classificador previamente treinado e salvar em um formato adequado para uso futuro. Em seguida, irei realizar a carga do modelo salvo e utilizar um conjunto de dados de teste para avaliar sua capacidade de previsão, especialmente em relação à identificação de funcionários propensos ao churn.

- ### Objetivos

1. Salvar o classificador com melhor desempenho obtido e documentado em outro notebook.

2. Carregar o modelo salvo e realizar a previsão com um conjunto de dados de teste.

3. Avaliar a capacidade do modelo em prever corretamente a maioria dos funcionários, com foco na análise de churn.

- ### Resultados Esperados

Espera-se que o modelo salvo apresente uma alta taxa de previsão correta, especialmente em relação aos funcionários propensos ao churn. Isso indicaria que o classificador está bem ajustado e pode ser usado com confiança para prever o churn de funcionários na empresa.

## Importação das bibliotecas, base de dados e EDA

- Base de dados: https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset


In [1]:
# Visualização e Manipulação dos Dados
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importa o módulo pickle para salvar e carregar objetos Python
import pickle

# Importa o modelo de regressão logística do scikit-learn
from sklearn.linear_model import LogisticRegression

# Importa a técnica SMOTE para oversampling de classes desbalanceadas
from imblearn.over_sampling import SMOTE

# Importa a função para dividir o conjunto de dados em treino e teste
from sklearn.model_selection import train_test_split

# Importa o MinMaxScaler para escalonar os dados
from sklearn.preprocessing import MinMaxScaler

# Importação das métricas de avaliação de modelos
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
)


# Importação da base de dados
base = pd.read_csv('/content/Human_Resources.csv')



In [2]:
# Transformando dados categóricos binários
base['Attrition'] = base['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
base['OverTime'] = base['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

# Excluindo as colunas que não irão influenciar na análise
base.drop(['EmployeeCount', 'StandardHours', 'Over18', 'EmployeeNumber'], axis=1, inplace=True)

# Importando o OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Lista das colunas a serem transformadas
columns_to_encode = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']

# Criando o encoder
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', dtype='int32')

# Ajustando o encoder para todas as colunas de uma vez
ohe.fit(base[columns_to_encode])

# Transformando as colunas
ohe_df = pd.DataFrame(ohe.transform(base[columns_to_encode]), columns=ohe.get_feature_names_out())
base = pd.concat([base, ohe_df], axis=1)
base.drop(columns=columns_to_encode, inplace=True)

# Separando previsores e a coluna alvo
X = base[['BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
       'BusinessTravel_Travel_Rarely', 'Department_Human Resources',
       'Department_Research & Development', 'Department_Sales',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Other', 'EducationField_Technical Degree',
       'Gender_Female', 'Gender_Male', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'JobRole_Sales Representative',
       'MaritalStatus_Divorced', 'MaritalStatus_Married',
       'MaritalStatus_Single','Age', 'DailyRate', 'DistanceFromHome',
       'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager']]

y = base['Attrition']

# Normalizando os dados
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
# Crie o modelo de Regressão Logística
rl = LogisticRegression(random_state=42)

# Crie o SMOTE para balanceamento das classes
smt = SMOTE(random_state=42)
X, y = smt.fit_resample(X, y)

# Separe em treino/teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X , y, test_size=0.25, random_state=42)

# Treine o modelo
rl.fit(X_treino, y_treino)


LogisticRegression(random_state=42)

## Salvando o Classificador

Aqui, estou salvando um modelo de regressão logística (rl), um scaler (scaler) e um encoder (ohe) em um arquivo chamado modelo_rl_smote.pkl. Em seguida, irei carregar esses objetos previamente salvos de volta em variáveis model, min_max e encoder para uso posterior.

In [5]:
# Salvando o modelo
with open('modelo_rl_smote.pkl', 'wb') as file:
    pickle.dump((rl, scaler, ohe), file)


## Testando o classificador

In [6]:
# Carregar o modelo, scaler e encoder previamente salvos
with open('modelo_rl_smote.pkl', 'rb') as file:
    model, min_max, encoder = pickle.load(file)

In [7]:
# Carregando o conjunto dados e vizualizando novamente
nova_base = pd.read_csv('/content/Human_Resources.csv')
nova_base.head(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
5,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,...,3,80,0,8,2,2,7,7,3,6
6,59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,1,80,3,12,3,2,1,0,0,0
7,30,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,...,2,80,1,1,2,3,1,0,0,0
8,38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,...,2,80,0,10,2,3,9,7,1,8
9,36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7


**OBS**: O Kaggle não disponibilizou um conjunto de dados desconhecido para teste, como para outras bases de dados existentes no site. Portanto, optarei por utilizar o conjunto de dados original para esta finalidade.

In [8]:
X_novo = nova_base.iloc[0:10]

# Transformando dados categóricos binários
X_novo['Attrition'] = X_novo['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
X_novo['OverTime'] = X_novo['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

#Excluindo colunas que não servem para nada
X_novo.drop(['EmployeeCount', 'StandardHours', 'Over18', 'EmployeeNumber'], axis=1, inplace=True)

X_cat_novo = X_novo[['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']]
X_cat_novo = encoder.transform(X_cat_novo)
X_cat_novo_df = pd.DataFrame(X_cat_novo, columns=encoder.get_feature_names_out())

X_numerical_novo = X_novo[['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']]

X_all_novo = pd.concat([X_cat_novo_df, X_numerical_novo], axis=1)

# Normalizando os dados
X_novo_scaled = min_max.transform(X_all_novo)


<ipython-input-8-3b31ff733b1b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_novo['Attrition'] = X_novo['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
<ipython-input-8-3b31ff733b1b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_novo['OverTime'] = X_novo['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
<ipython-input-8-3b31ff733b1b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [9]:
# Realizando a predição
previsoes = model.predict(X_novo_scaled)
probabilidades = model.predict_proba(X_novo_scaled)

# Exibindo resultados de forma mais bonita
print("Previsões:")
for i, pred in enumerate(previsoes):
    print(f"Exemplo {i}: {pred} (Probabilidades: {probabilidades[i]})")

Previsões:
Exemplo 0: 1 (Probabilidades: [0.09643746 0.90356254])
Exemplo 1: 0 (Probabilidades: [0.92287348 0.07712652])
Exemplo 2: 1 (Probabilidades: [0.19677261 0.80322739])
Exemplo 3: 0 (Probabilidades: [0.59371409 0.40628591])
Exemplo 4: 1 (Probabilidades: [0.24818449 0.75181551])
Exemplo 5: 0 (Probabilidades: [0.74010229 0.25989771])
Exemplo 6: 0 (Probabilidades: [0.51826554 0.48173446])
Exemplo 7: 0 (Probabilidades: [0.83684316 0.16315684])
Exemplo 8: 0 (Probabilidades: [0.74433658 0.25566342])
Exemplo 9: 0 (Probabilidades: [0.74389966 0.25610034])


### Relatório de Previsões:

Este relatório apresenta as previsões de um modelo para um conjunto de dados. Abaixo estão as previsões para os primeiros 10 exemplos do conjunto de dados, juntamente com as probabilidades associadas a cada classe:

1. **Exemplo 0 (Resultado Real = 1):** Previsão - 1; Probabilidades - [0.09643746, 0.90356254]
2. **Exemplo 1 (Resultado Real = 0):** Previsão - 0; Probabilidades - [0.92287348, 0.07712652]
3. **Exemplo 2 (Resultado Real = 1):** Previsão - 1; Probabilidades - [0.19677261, 0.80322739]
4. **Exemplo 3 (Resultado Real = 0):** Previsão - 0; Probabilidades - [0.59371409, 0.40628591]
5. **Exemplo 4 (Resultado Real = 1):** Previsão - 1; Probabilidades - [0.24818449, 0.75181551]
6. **Exemplo 5 (Resultado Real = 0):** Previsão - 0; Probabilidades - [0.74010229, 0.25989771]
7. **Exemplo 6 (Resultado Real = 0):** Previsão - 0; Probabilidades - [0.51826554, 0.48173446]
8. **Exemplo 7 (Resultado Real = 0):** Previsão - 0; Probabilidades - [0.83684316, 0.16315684]
9. **Exemplo 8 (Resultado Real = 0):** Previsão - 0; Probabilidades - [0.74433658, 0.25566342]
10. **Exemplo 9 (Resultado Real = 0):** Previsão - 0; Probabilidades - [0.74389966, 0.25610034]

Analisando os resultados, é possível observar que, para essa pequena amostragem o modelo apresenta um desempenho satisfatório na previsão dos dados. Dos 10 exemplos apresentados, o modelo acertou 10 previsões, demonstrando uma taxa de acerto de 100%. É relevante destacar que, dentre os casos de churn presentes na amostra, o modelo foi capaz de acertar todos, indicando uma capacidade adequada de identificação desses casos. Esses resultados sugerem que o modelo pode ser eficaz na previsão de churn em um contexto mais amplo.

Então, o modelo de **Regressão Logística com SMOTE**, pode ser aplicado para casos reais.
